In [11]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import numpy as np
import scipy.stats as st
import pandas as pd
from scipy.stats import f_oneway
import re

# 그래프 설정
sns.set()

# 그래프 기본 설정
# plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 결측치 시각화를 위한 라이브러리
import missingno

### 데이터를 불러온다.

In [14]:
df1 = pd.read_parquet('open/test/7.마케팅정보/201807_test_마케팅정보.parquet')
df2 = pd.read_parquet('open/test/7.마케팅정보/201808_test_마케팅정보.parquet')
df3 = pd.read_parquet('open/test/7.마케팅정보/201809_test_마케팅정보.parquet')
df4 = pd.read_parquet('open/test/7.마케팅정보/201810_test_마케팅정보.parquet')
df5 = pd.read_parquet('open/test/7.마케팅정보/201811_test_마케팅정보.parquet')
df6 = pd.read_parquet('open/test/7.마케팅정보/201812_test_마케팅정보.parquet')

### 데이터를 합친다.

In [17]:
df7 = pd.concat([df1, df2, df3, df4, df5, df6], axis=0, ignore_index=True)

In [19]:
df7

,기준년월,ID,컨택건수_카드론_TM_B0M,컨택건수_리볼빙_TM_B0M,컨택건수_CA_TM_B0M,컨택건수_이용유도_TM_B0M,컨택건수_신용발급_TM_B0M,컨택건수_부대서비스_TM_B0M,컨택건수_포인트소진_TM_B0M,컨택건수_보험_TM_B0M,...,컨택건수_카드론_당사앱_R6M,컨택건수_CA_당사앱_R6M,컨택건수_리볼빙_당사앱_R6M,컨택건수_이용유도_당사앱_R6M,컨택건수_채권_B0M,컨택건수_FDS_B0M,컨택건수_채권_R6M,컨택건수_FDS_R6M,캠페인접촉건수_R12M,캠페인접촉일수_R12M
0,201807,TEST_00000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1회 이상,1일 이상
1,201807,TEST_00001,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5회 이상,5일 이상
2,201807,TEST_00002,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10회 이상,10일 이상
3,201807,TEST_00003,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15회 이상,15일 이상
4,201807,TEST_00004,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1회 이상,1일 이상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1회 이상,1일 이상
599996,201812,TEST_99996,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1회 이상,1일 이상
599997,201812,TEST_99997,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,1회 이상,1일 이상
599998,201812,TEST_99998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5회 이상,5일 이상


### train 데이터에 남은 컬럼들만 남긴다

In [21]:
cols_to_save = ['기준년월', 'ID','컨택건수_이용유도_TM_R6M','컨택건수_이용유도_EM_R6M','컨택건수_이용유도_EM_B0M','컨택건수_이용유도_청구서_B0M',
'컨택건수_이용유도_TM_B0M','컨택건수_이용유도_인터넷_R6M','컨택건수_신용발급_TM_R6M','컨택건수_이용유도_LMS_B0M','컨택건수_이용유도_청구서_R6M',
'컨택건수_포인트소진_TM_R6M', '캠페인접촉건수_R12M', '캠페인접촉일수_R12M']

In [23]:
df8 = df7[cols_to_save].copy()
df8

,기준년월,ID,컨택건수_이용유도_TM_R6M,컨택건수_이용유도_EM_R6M,컨택건수_이용유도_EM_B0M,컨택건수_이용유도_청구서_B0M,컨택건수_이용유도_TM_B0M,컨택건수_이용유도_인터넷_R6M,컨택건수_신용발급_TM_R6M,컨택건수_이용유도_LMS_B0M,컨택건수_이용유도_청구서_R6M,컨택건수_포인트소진_TM_R6M,캠페인접촉건수_R12M,캠페인접촉일수_R12M
0,201807,TEST_00000,0,2,1,0,0,4,0,2,0,0,1회 이상,1일 이상
1,201807,TEST_00001,2,3,0,0,0,1,0,0,0,0,5회 이상,5일 이상
2,201807,TEST_00002,2,5,1,0,0,2,0,2,2,0,10회 이상,10일 이상
3,201807,TEST_00003,3,22,0,0,0,6,0,0,0,0,15회 이상,15일 이상
4,201807,TEST_00004,0,9,0,0,0,2,0,2,1,0,1회 이상,1일 이상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,0,0,0,0,0,0,0,0,0,0,1회 이상,1일 이상
599996,201812,TEST_99996,0,9,2,0,0,3,0,0,0,0,1회 이상,1일 이상
599997,201812,TEST_99997,18,12,1,0,3,1,0,2,0,0,1회 이상,1일 이상
599998,201812,TEST_99998,0,0,0,1,0,0,0,0,1,0,5회 이상,5일 이상


### 범주형 독립변수 인코딩

In [25]:
import re

# 0) 데이터 복사 ─ 원본 보존
# -------------------------------------
df9 = df8.copy()          # 원본 df는 그대로 유지

ORD_COLS = ['캠페인접촉건수_R12M', '캠페인접촉일수_R12M']

def ordinal_encode_inplace(df9, col):
    """숫자값 기준으로 오름차순 정렬 → 0,1,2… 로 해당 컬럼을 덮어쓰기"""
    # 1) 고유 카테고리 추출
    uniq = df9[col].dropna().unique()
    
    # 2) 숫자만 뽑아 정렬
    sort_key = lambda x: int(re.search(r'\d+', str(x)).group())
    ordered = sorted(uniq, key=sort_key)          # ['1회 이상', '5회 이상', …]
    
    # 3) 매핑 딕셔너리 생성
    mapping = {cat: idx for idx, cat in enumerate(ordered)}
    print(f"[{col}] 매핑:", mapping)
    
    # 4) **기존 컬럼에 바로 덮어쓰기**
    df9[col] = df9[col].map(mapping).astype('int8')

# ---------- 실제 적용 ----------
for c in ORD_COLS:
    ordinal_encode_inplace(df9, c)

[캠페인접촉건수_R12M] 매핑: {'1회 이상': 0, '5회 이상': 1, '10회 이상': 2, '15회 이상': 3, '20회 이상': 4, '25회 이상': 5}
[캠페인접촉일수_R12M] 매핑: {'1일 이상': 0, '5일 이상': 1, '10일 이상': 2, '15일 이상': 3, '20일 이상': 4}


In [27]:
df9

,기준년월,ID,컨택건수_이용유도_TM_R6M,컨택건수_이용유도_EM_R6M,컨택건수_이용유도_EM_B0M,컨택건수_이용유도_청구서_B0M,컨택건수_이용유도_TM_B0M,컨택건수_이용유도_인터넷_R6M,컨택건수_신용발급_TM_R6M,컨택건수_이용유도_LMS_B0M,컨택건수_이용유도_청구서_R6M,컨택건수_포인트소진_TM_R6M,캠페인접촉건수_R12M,캠페인접촉일수_R12M
0,201807,TEST_00000,0,2,1,0,0,4,0,2,0,0,0,0
1,201807,TEST_00001,2,3,0,0,0,1,0,0,0,0,1,1
2,201807,TEST_00002,2,5,1,0,0,2,0,2,2,0,2,2
3,201807,TEST_00003,3,22,0,0,0,6,0,0,0,0,3,3
4,201807,TEST_00004,0,9,0,0,0,2,0,2,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,0,0,0,0,0,0,0,0,0,0,0,0
599996,201812,TEST_99996,0,9,2,0,0,3,0,0,0,0,0,0
599997,201812,TEST_99997,18,12,1,0,3,1,0,2,0,0,0,0
599998,201812,TEST_99998,0,0,0,1,0,0,0,0,1,0,1,1


### Segment 컬럼을 추가하고 결측치로 채운다.

In [30]:
df9.insert(
    loc=2,                # 세 번째 칼럼 자리 (0,1,**2**)
    column='Segment',     # 새 컬럼 이름
    value=np.nan          # 결측치로 채우기 (또는 pd.NA)
)

In [32]:
df9

,기준년월,ID,Segment,컨택건수_이용유도_TM_R6M,컨택건수_이용유도_EM_R6M,컨택건수_이용유도_EM_B0M,컨택건수_이용유도_청구서_B0M,컨택건수_이용유도_TM_B0M,컨택건수_이용유도_인터넷_R6M,컨택건수_신용발급_TM_R6M,컨택건수_이용유도_LMS_B0M,컨택건수_이용유도_청구서_R6M,컨택건수_포인트소진_TM_R6M,캠페인접촉건수_R12M,캠페인접촉일수_R12M
0,201807,TEST_00000,NaN,0,2,1,0,0,4,0,2,0,0,0,0
1,201807,TEST_00001,NaN,2,3,0,0,0,1,0,0,0,0,1,1
2,201807,TEST_00002,NaN,2,5,1,0,0,2,0,2,2,0,2,2
3,201807,TEST_00003,NaN,3,22,0,0,0,6,0,0,0,0,3,3
4,201807,TEST_00004,NaN,0,9,0,0,0,2,0,2,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,NaN,0,0,0,0,0,0,0,0,0,0,0,0
599996,201812,TEST_99996,NaN,0,9,2,0,0,3,0,0,0,0,0,0
599997,201812,TEST_99997,NaN,18,12,1,0,3,1,0,2,0,0,0,0
599998,201812,TEST_99998,NaN,0,0,0,1,0,0,0,0,1,0,1,1


In [34]:
df9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   기준년월               600000 non-null  int64  
 1   ID                 600000 non-null  object 
 2   Segment            0 non-null       float64
 3   컨택건수_이용유도_TM_R6M   600000 non-null  int64  
 4   컨택건수_이용유도_EM_R6M   600000 non-null  int64  
 5   컨택건수_이용유도_EM_B0M   600000 non-null  int64  
 6   컨택건수_이용유도_청구서_B0M  600000 non-null  int64  
 7   컨택건수_이용유도_TM_B0M   600000 non-null  int64  
 8   컨택건수_이용유도_인터넷_R6M  600000 non-null  int64  
 9   컨택건수_신용발급_TM_R6M   600000 non-null  int64  
 10  컨택건수_이용유도_LMS_B0M  600000 non-null  int64  
 11  컨택건수_이용유도_청구서_R6M  600000 non-null  int64  
 12  컨택건수_포인트소진_TM_R6M  600000 non-null  int64  
 13  캠페인접촉건수_R12M       600000 non-null  int8   
 14  캠페인접촉일수_R12M       600000 non-null  int8   
dtypes: float64(1), int64(11), int8(2), object(1)
memory

In [154]:
df9.to_parquet('open/test/7.마케팅정보/test_마케팅정보.parquet', index=False)

In [36]:
df9

,기준년월,ID,Segment,컨택건수_이용유도_TM_R6M,컨택건수_이용유도_EM_R6M,컨택건수_이용유도_EM_B0M,컨택건수_이용유도_청구서_B0M,컨택건수_이용유도_TM_B0M,컨택건수_이용유도_인터넷_R6M,컨택건수_신용발급_TM_R6M,컨택건수_이용유도_LMS_B0M,컨택건수_이용유도_청구서_R6M,컨택건수_포인트소진_TM_R6M,캠페인접촉건수_R12M,캠페인접촉일수_R12M
0,201807,TEST_00000,NaN,0,2,1,0,0,4,0,2,0,0,0,0
1,201807,TEST_00001,NaN,2,3,0,0,0,1,0,0,0,0,1,1
2,201807,TEST_00002,NaN,2,5,1,0,0,2,0,2,2,0,2,2
3,201807,TEST_00003,NaN,3,22,0,0,0,6,0,0,0,0,3,3
4,201807,TEST_00004,NaN,0,9,0,0,0,2,0,2,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,NaN,0,0,0,0,0,0,0,0,0,0,0,0
599996,201812,TEST_99996,NaN,0,9,2,0,0,3,0,0,0,0,0,0
599997,201812,TEST_99997,NaN,18,12,1,0,3,1,0,2,0,0,0,0
599998,201812,TEST_99998,NaN,0,0,0,1,0,0,0,0,1,0,1,1
